# 📢 데이터셋 구축

## 📌 라이브러리 설치 및 경로 변경 작업
* <b>"📌 스타일 단위 크롤링"</b> 에서 라이브러리 import가 잘 된다면 생략 가능.

In [1]:
# ! git clone https://github.com/pyg-team/pytorch_geometric.git

Cloning into 'pytorch_geometric'...
remote: Enumerating objects: 64805, done.
remote: Counting objects: 100% (2338/2338), done.
remote: Compressing objects: 100% (532/532), done.
remote: Total 64805 (delta 2085), reused 1931 (delta 1804), pack-reused 62467
Receiving objects: 100% (64805/64805), 15.55 MiB | 17.08 MiB/s, done.
Resolving deltas: 100% (50152/50152), done.


In [2]:
! cp ./bottom_to_upper.py ./pytorch_geometric/torch_geometric/datasets/bottom_to_upper.py
! cp ./upper_to_bottom.py ./pytorch_geometric/torch_geometric/datasets/upper_to_bottom.py
! cp ./__init__.py ./pytorch_geometric/torch_geometric/datasets/__init__.py

파이썬 site-packages 경로에 복사합니다

⬇⬇⬇ 아래는 Docker DSfinal 환경 copy

In [4]:
!cp -r /step1/source/pytorch_geometric/torch_geometric /opt/conda/lib/python3.10/site-packages/

⬇⬇⬇ 아래는 윈도우 환경 copy 예시

In [11]:
#! cp -r ./pytorch_geometric/torch_geometric 'c:\\Users\\AMD\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages'

Active code page: 65001


현재 위치의 geometric 삭제(선택)

In [5]:
! rm -r ./pytorch_geometric

### 🍔커널 <b>재시작</b> 필요합니다.

## 📌 스타일 단위 크롤링
* 코디샵의 개별아이템 상세페이지 링크 수집
* 개별아이템 상세페이지 링크 실행하며 스타일, 태그값 등을 수집하여 데이터셋 구축
* 팀원들간 개별아이템 수집일이 다를 수 있지만, ITEM_ID기준으로 중복제거 및 nan값 제거 가능
* lightGCN 학습을 위해, 전문코디의 스타일(상,하의 묶음)을 CODI_ID로 지정하여 데이터셋 구축 

In [1]:
import platform

if platform.system() == 'Windows':
    print(f"\x1b[32m\" 윈도우 환경 작동 중... \"\x1b[0m")
    import selenium
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.service import Service #이거 추가
    import requests
    from bs4 import BeautifulSoup
    from selenium.webdriver.remote.webelement import WebElement
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import WebDriverException
else:
    print(f"\x1b[32m\" 리눅스 계열 작동 중... \"\x1b[0m")


import pandas as pd
import os
import time
import random
from tqdm import tqdm
import copy
from pprint import pprint
import re
import numpy as np
import sqlite3
from collections import Counter
from datetime import datetime
import torch

import os.path as osp
from torch_geometric.datasets import BottomToUpper
from torch_geometric.datasets import UpperToBottom
from torch_geometric.nn import LightGCN
from torch_geometric.utils import degree



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10000)

print(f"현재 사용 중인 디바이스: {device}")

driver = None

" 리눅스 계열 작동 중... "


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


현재 사용 중인 디바이스: cuda:0


In [2]:
CODI_BASE_URL = {
                    'americancasual': 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=americancasual&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'casual' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=casual&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'chic' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=chic&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'dandy' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=dandy&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'formal' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=formal&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'girlish' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=girlish&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'golf' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=golf&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'retro' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=retro&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'romantic' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=romantic&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'sports' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=sports&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'street' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=street&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=',
                    'gorpcore' : 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=gorpcore&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page='
                }
for idx, item in enumerate(CODI_BASE_URL):
    print(f'{idx+1}번: {CODI_BASE_URL[item]}')

1번: https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=americancasual&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=
2번: https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=casual&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=
3번: https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=chic&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=
4번: https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=dandy&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=
5번: https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=formal&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=
6번: https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=girlish&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page=
7번: https://www.musinsa.com/app/styles/lists?u

In [3]:
def chrome_settings(_show):
    global driver
    chrome_options = webdriver.ChromeOptions()
    
    if not _show:
        chrome_options.add_argument(argument='--headless')
        
    chrome_options.add_argument(argument='--no-sandbox')
    chrome_options.add_argument(argument='--disable-dev-shm-usage')
    chrome_options.add_argument(argument="--disable-infobars")
    chrome_options.add_argument(argument="--disable-notifications")

    if platform.system() == 'Windows':
        print("윈도우 환경 작동 중..")
        service = Service(executable_path=os.path.join("..", "..", "chromedriver.exe"))
    else:
        print("리눅스 계열 작동 중..")
        service = Service(executable_path=os.path.join("..", "..", "chromedriver"))

    if selenium.__version__[0] == '4':
        driver = webdriver.Chrome(service=service, options=chrome_options)
    else:
        if platform.system() == 'Windows':
            driver = webdriver.Chrome(executable_path=os.path.join("..", "..", "chromedriver.exe"), options=chrome_options)
        else:
            driver = webdriver.Chrome(executable_path=os.path.join("..", "..", "chromedriver"), options=chrome_options)
    driver.maximize_window()
    driver.implicitly_wait(30)

### 🍔 실행 (1/3 단계)
* 크롤링 진행하는 경우 A 루트
* csv 파일 있는 경우 B 루트

#### 🍟A 루트 (크롤링)

In [28]:
before_df_list = list()
for idx, item in enumerate(CODI_BASE_URL.values()):
    print(f"\x1b[33m\" {idx+1}번: {list(CODI_BASE_URL.keys())[idx]} 진행 중...\"\x1b[0m")
    chrome_settings(True) # True하면 크롬브라우저 뜸
    tmp_list = list()
    try:
        driver.get(item + str(1))
        time.sleep(random.uniform(2, 3))

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_count = int(soup.prettify().split('totalPagingNum">')[1].split('</span>')[0].strip())

        for i in tqdm(range(page_count)):
            if i != 0:
                driver.get(item + str(i+1))
                time.sleep(random.uniform(2, 3))
            tmp_soup = BeautifulSoup(driver.page_source, 'html.parser')
            for j, item2 in enumerate(tmp_soup.find_all(class_ = 'style-list-item')):
                match = re.search(r'\d+', item2.a['onclick']) # 정규표현식: goView(코디링크ID) 숫자만 추출
                if match:
                    extracted_number = match.group()
                    if str(item2.find_all('span')[1].text) == 'N':
                        tmp_list.append([extracted_number, str(i+1), item2.span.text, item2.find_all('span')[2].text, item2.find_all('span')[3].text, item2.strong.text, item2.find(class_='style-list-model').text, item2.find('img').get('src')])
                    else:
                        tmp_list.append([extracted_number, str(i+1), item2.span.text, item2.find_all('span')[1].text, item2.find_all('span')[2].text, item2.strong.text, item2.find(class_='style-list-model').text, item2.find('img').get('src')])
    finally:
        driver.close()
    before_df_list.append(tmp_list)
    pd.DataFrame(tmp_list, columns=['CODI_ID', 'PAGE_NUM', 'STYLE', 'DATE', 'VIEWS', 'TAG1', 'MODEL', 'CODI_IMG']).to_csv(os.path.join("..", "data", "_temp", f"{list(CODI_BASE_URL.keys())[idx]}.csv"), index=True, encoding='utf-8')
    print(f"\x1b[32m\"{idx+1}번: {list(CODI_BASE_URL.keys())[idx]} 완료...\"\x1b[0m")
    display(pd.DataFrame(tmp_list, columns=['CODI_ID', 'PAGE_NUM', 'STYLE', 'DATE', 'VIEWS', 'TAG1', 'MODEL', 'CODI_IMG']).tail(1))

df = pd.DataFrame()
for i in before_df_list:
    df = pd.concat([df, pd.DataFrame(i, columns=['CODI_ID', 'PAGE_NUM', 'STYLE', 'DATE', 'VIEWS', 'TAG1', 'MODEL', 'CODI_IMG'])], ignore_index=True)
df['LEVEL'] = 0
df.replace('아메리칸 캐주얼', 'americancasual', inplace=True) # 이 부분.. STYLE 컬럼에서만 적용하는 코드로 바꿔야함. - 상현님, 해지님 도움 필요
df.replace('캐주얼', 'casual', inplace=True)
df.replace('시크', 'chic', inplace=True)
df.replace('댄디', 'dandy', inplace=True)
df.replace('포멀', 'formal', inplace=True)
df.replace('걸리시', 'girlish', inplace=True)
df.replace('골프', 'golf', inplace=True)
df.replace('레트로', 'retro', inplace=True)
df.replace('로맨틱', 'romantic', inplace=True)
df.replace('스포츠', 'sports', inplace=True)
df.replace('스트릿', 'street', inplace=True)
df.replace('고프코어', 'gorpcore', inplace=True)
df['CODI_URL'] = 'https://www.musinsa.com/app/styles/views/' + df['CODI_ID'] + '?use_yn_360=&style_type='+ df['STYLE'] +'&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page='+ df['PAGE_NUM']
df.to_csv(os.path.join("..", "data", "codishop_temp_all.csv"), index=True, encoding='utf-8')
display(df.tail(1))

" 1번: americancasual 진행 중..."
윈도우 환경 작동 중..


100%|██████████| 6/6 [00:17<00:00,  2.98s/it]

"1번: americancasual 완료..."


,CODI_ID,PAGE_NUM,STYLE,DATE,VIEWS,TAG1,MODEL,CODI_IMG
305,23923,6,아메리칸 캐주얼,21.11.25,"조회 1,947",센스 더하기,모델 : 박호영,//image.msscdn.net/images/style/list/l_3_20211...


" 2번: casual 진행 중..."
윈도우 환경 작동 중..


...

" 12번: gorpcore 진행 중..."
윈도우 환경 작동 중..


100%|██████████| 5/5 [00:12<00:00,  2.50s/it]

"12번: gorpcore 완료..."


,CODI_ID,PAGE_NUM,STYLE,DATE,VIEWS,TAG1,MODEL,CODI_IMG
288,20947,5,고프코어,21.05.04,"조회 2,959",트레킹 룩에 제격,모델 : 임재완,//image.msscdn.net/images/style/list/l_3_20210...


,CODI_ID,PAGE_NUM,STYLE,DATE,VIEWS,TAG1,MODEL,CODI_IMG,LEVEL,CODI_URL
30763,20947,5,gorpcore,21.05.04,"조회 2,959",트레킹 룩에 제격,모델 : 임재완,//image.msscdn.net/images/style/list/l_3_20210...,0,https://www.musinsa.com/app/styles/views/20947...


#### 🍟B 루트 (파일 읽어오기)

In [2]:
df = pd.read_csv(os.path.join("..", "data" ,"codishop_temp_all.csv"), index_col=0)
print(df.shape)
df.tail(5)

(30764, 10)


,CODI_ID,PAGE_NUM,STYLE,DATE,VIEWS,TAG1,MODEL,CODI_IMG,LEVEL,CODI_URL
30759,21396,5,gorpcore,21.06.10,"조회 1,739",스포츠 즐기기,모델 : 함재민,//image.msscdn.net/images/style/list/l_3_20210...,0,https://www.musinsa.com/app/styles/views/21396...
30760,21323,5,gorpcore,21.06.08,"조회 1,661",캠핑 가자!,모델 : 박준성,//image.msscdn.net/images/style/list/l_3_20210...,0,https://www.musinsa.com/app/styles/views/21323...
30761,21185,5,gorpcore,21.05.28,"조회 1,610",가벼운 스포츠 룩,모델 : 이지한,//image.msscdn.net/images/style/list/l_3_20210...,0,https://www.musinsa.com/app/styles/views/21185...
30762,21087,5,gorpcore,21.05.21,"조회 1,751",스포츠 즐기기,모델 : 최규민,//image.msscdn.net/images/style/list/l_3_20210...,0,https://www.musinsa.com/app/styles/views/21087...
30763,20947,5,gorpcore,21.05.04,"조회 2,959",트레킹 룩에 제격,모델 : 임재완,//image.msscdn.net/images/style/list/l_3_20210...,0,https://www.musinsa.com/app/styles/views/20947...


### 🍔 실행 (2/3)

#### 🍟 A 루트

##### ☕ A - 1 크롤링

In [ ]:
before_df_list = list()
tmp_list = list()
chrome_settings(True) # True하면 크롬브라우저 뜸
try:
    home_url = 'https://www.musinsa.com'
    driver.get(home_url)
    print(f"현재 URL: {driver.current_url}\nPage title: {driver.title}")

    for idx, url in tqdm(enumerate(df['CODI_URL'])):
        driver.execute_script(f"window.open('{url}', '_blank')")
        time.sleep(random.uniform(1, 2))
        driver.switch_to.window(driver.window_handles[1])

        # 로직
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        for i in soup.find_all('a', class_='styling_img'):
            tmp_list.append([home_url + i['href'], url]) # 아이템 url, 코디 url

        driver.close()
        time.sleep(random.uniform(1, 2))
        driver.switch_to.window(driver.window_handles[0])
finally:
    driver.close()

df_tmp = pd.DataFrame(tmp_list, columns=['ITEM', 'CODI_URL'])
df_tmp['ITEM_ID'] = df_tmp['ITEM'].apply(lambda x: x.split('/')[-1])
df_tmp['CODI_ID'] = df_tmp['CODI_URL'].apply(lambda x: x.split('?')[0])
df_tmp['CODI_ID'] = df_tmp['CODI_ID'].apply(lambda x: x.split('/')[-1])
df_tmp.insert(0, 'ITEM_ID', df_tmp.pop('ITEM_ID'))
df_tmp.insert(1, 'CODI_ID', df_tmp.pop('CODI_ID'))
df_tmp['LEVEL'] = 1
df_tmp.insert(0, 'LEVEL', df_tmp.pop('LEVEL'))
df_tmp['STYLE'] = np.nan
df_tmp.insert(3, 'STYLE', df_tmp.pop('STYLE'))
df_tmp.insert(4, 'DATE', np.nan)
df_tmp.insert(5, 'VIEWS', np.nan)
df_tmp.insert(6, 'TAG1', np.nan)
df_tmp.insert(7, 'MODEL', np.nan)
df_tmp.insert(9, 'ITEM_URL', df_tmp.pop('ITEM'))
df_tmp.to_csv(os.path.join("..", "data" ,"codishop_item.csv"), index=True, encoding='utf-8')
df_tmp

##### ☕ A - 2 전처리 (B루트와 동일함)

In [ ]:
df.insert(0, 'LEVEL', df.pop('LEVEL'))
df.insert(1, 'ITEM_ID', np.nan)
df['ITEM_ID'].astype('object')
df.drop(['PAGE_NUM', 'CODI_IMG'], axis=1, inplace=True)

df_tmp = pd.read_csv(os.path.join("..", "data" ,"codishop_item.csv"), index_col=0)
df_tmp['ITEM_ID'] = df_tmp['ITEM_ID'].astype('object')


df_concat = pd.concat([df, df_tmp], ignore_index=True, axis=0)

tmp_list = list()
print(f"\x1b[33m\"LEVEL 1 데이터 STYLE 타입 처리...\"\x1b[0m")
for i in tqdm(range(len(df_concat[df_concat['LEVEL'] == 1]))):
    tmp_list.append(df_concat['CODI_URL'][i].split('style_type=')[1].split('&')[0])

tmp_se = pd.Series(tmp_list)

tmp_index = 0
for idx, item in tqdm(enumerate(df_concat['STYLE'])):
    if item is np.nan:
        df_concat.iloc[idx, 3] = tmp_se[tmp_index]
        tmp_index += 1
print("\x1b[32m\"LEVEL 1 데이터 STYLE 타입 처리 완료\"\x1b[0m")

df_level0 = df_concat[df_concat['LEVEL'] == 0]
df_level1 = df_concat[df_concat['LEVEL'] == 1]

print(f"\x1b[33m\"LEVEL 0 데이터 ITEM_ID 처리...\"\x1b[0m")
tmp_dict = dict()
for codi, item in tqdm(zip(df_concat['CODI_ID'], df_concat['ITEM_ID'])):
    if item is not np.nan:
        if codi in list(tmp_dict.keys()): 
            tmp_dict[codi] = tmp_dict[codi] + ',' + str(item)
        else:
            tmp_dict[codi] = str(item)
dict_key = list(tmp_dict.keys())
for idx, item in enumerate(df_concat['CODI_ID']):
    if item in dict_key and df_concat['ITEM_ID'][idx] is np.nan:
        df_concat.iloc[idx, 1] = tmp_dict[item]
print("\x1b[32m\"LEVEL 0 데이터 ITEM_ID 처리 완료\"\x1b[0m")
print(df_concat.isna().sum())
display(df_concat)

#### 🍟 B 루트 (전처리)

In [3]:
df.insert(0, 'LEVEL', df.pop('LEVEL'))
df.insert(1, 'ITEM_ID', np.nan)
df['ITEM_ID'].astype('object')
df.drop(['PAGE_NUM', 'CODI_IMG'], axis=1, inplace=True)

df_tmp = pd.read_csv(os.path.join("..", "data" ,"codishop_item.csv"), index_col=0)
df_tmp['ITEM_ID'] = df_tmp['ITEM_ID'].astype('object')


df_concat = pd.concat([df, df_tmp], ignore_index=True, axis=0)

tmp_list = list()
print(f"\x1b[33m\"LEVEL 1 데이터 STYLE 타입 처리...\"\x1b[0m")
for i in tqdm(range(len(df_concat[df_concat['LEVEL'] == 1]))):
    tmp_list.append(df_concat['CODI_URL'][i].split('style_type=')[1].split('&')[0])

tmp_se = pd.Series(tmp_list)

tmp_index = 0
for idx, item in tqdm(enumerate(df_concat['STYLE'])):
    if item is np.nan:
        df_concat.iloc[idx, 3] = tmp_se[tmp_index]
        tmp_index += 1
print("\x1b[32m\"LEVEL 1 데이터 STYLE 타입 처리 완료\"\x1b[0m")

df_level0 = df_concat[df_concat['LEVEL'] == 0]
df_level1 = df_concat[df_concat['LEVEL'] == 1]

print(f"\x1b[33m\"LEVEL 0 데이터 ITEM_ID 처리...\"\x1b[0m")
tmp_dict = dict()
for codi, item in tqdm(zip(df_concat['CODI_ID'], df_concat['ITEM_ID'])):
    if item is not np.nan:
        if codi in list(tmp_dict.keys()): 
            tmp_dict[codi] = tmp_dict[codi] + ',' + str(item)
        else:
            tmp_dict[codi] = str(item)
dict_key = list(tmp_dict.keys())
for idx, item in enumerate(df_concat['CODI_ID']):
    if item in dict_key and df_concat['ITEM_ID'][idx] is np.nan:
        df_concat.iloc[idx, 1] = tmp_dict[item]
print("\x1b[32m\"LEVEL 0 데이터 ITEM_ID 처리 완료\"\x1b[0m")
print(df_concat.isna().sum())
display(df_concat)

"LEVEL 1 데이터 STYLE 타입 처리..."


100%|██████████| 98521/98521 [00:00<00:00, 417259.42it/s]
129285it [00:09, 13319.77it/s]


"LEVEL 1 데이터 STYLE 타입 처리 완료"
"LEVEL 0 데이터 ITEM_ID 처리..."


129285it [00:06, 19839.92it/s]


"LEVEL 0 데이터 ITEM_ID 처리 완료"
LEVEL           0
ITEM_ID      3800
CODI_ID         0
STYLE           0
DATE        98521
VIEWS       98521
TAG1        98521
MODEL       98521
CODI_URL        0
ITEM_URL    30764
dtype: int64


,LEVEL,ITEM_ID,CODI_ID,STYLE,DATE,VIEWS,TAG1,MODEL,CODI_URL,ITEM_URL
0,0,"3450087,3540430,3227713,3035852,3429258",35815,americancasual,23.09.27,조회 607,스포티 바이브,모델 : 송혜원,https://www.musinsa.com/app/styles/views/35815...,NaN
1,0,"3451226,3524050,3523872,3431905,3450111,3450134",36233,americancasual,23.09.27,조회 587,시티보이 룩,모델 : 진킴,https://www.musinsa.com/app/styles/views/36233...,NaN
2,0,"3484627,3493988,3453009,3413269,3453042,3463945",36228,americancasual,23.09.27,조회 839,americancasual,모델 : 진킴,https://www.musinsa.com/app/styles/views/36228...,NaN
3,0,"3153140,3457629,3542129,2247148,3461469,3497070",36082,americancasual,23.09.21,조회 430,오버올 스타일,모델 : 김진령,https://www.musinsa.com/app/styles/views/36082...,NaN
4,0,"3561777,3483411,3430632,3152009,3431911,3479540",36081,americancasual,23.09.21,조회 925,프레피 룩,모델 : 김진령,https://www.musinsa.com/app/styles/views/36081...,NaN
...,...,...,...,...,...,...,...,...,...,...
129280,1,1290270,21087,girlish,NaN,NaN,NaN,NaN,https://www.musinsa.com/app/styles/views/21087...,https://www.musinsa.com/app/goods/1290270
129281,1,1771289,21087,girlish,NaN,NaN,NaN,NaN,https://www.musinsa.com/app/styles/views/21087...,https://www.musinsa.com/app/goods/1771289
129282,1,1771276,20947,girlish,NaN,NaN,NaN,NaN,https://www.musinsa.com/app/styles/views/20947...,https://www.musinsa.com/app/goods/1771276
129283,1,1866392,20947,girlish,NaN,NaN,NaN,NaN,https://www.musinsa.com/app/styles/views/20947...,https://www.musinsa.com/app/goods/1866392


### 🍔 실행 (3/3)
* 추후에 실행 2/3과 통합 예정
* 다른 팀원의 db를 사용해서 빠르게 처리하기 위한 코드입니다.

#### 🍟 임시 통합본(1/2)

In [4]:
conn = sqlite3.connect(os.path.join("..", "data" , "musinsa_codishop.db"))

cur = conn.cursor()

query = """
SELECT * 
FROM outfit_data
"""

cur.execute(query)

results = cur.fetchall()

df_db = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])

conn.close()
df_db.tail(1)

df_db.insert(0, 'ITEM_ID', df_db.pop('page_number'))
df_db.drop_duplicates(subset=['ITEM_ID'], keep='first', inplace=True)
df_db.reset_index(drop=True, inplace=True)

df_level1_db = pd.merge(df_level1, df_db, how='left', on='ITEM_ID')
df_level1_db['TAG1'] = df_level1_db['category_1']
df_level1_db['DATE'] =  df_level1_db['codi_upload_date']
df_level1_db['VIEWS'] = df_level1_db['codi_number_of_views']
df_level1_db.insert(7, 'TAG2', df_level1_db['category_2'])
df_level1_db.insert(8, 'GENDER', df_level1_db['gender'])

print(df_level1_db.isna().sum())
df_level1_db.drop(['id', 'category_1', 'category_2', 'brand', 'gender', 'outfit_tags', 'img_url', 'rating', 'codi_number', 'codi_tags', 'codi_upload_date', 'codi_number_of_views'], axis=1, inplace=True)
display(df_level1_db)



LEVEL                       0
ITEM_ID                     0
CODI_ID                     0
STYLE                       0
DATE                      567
VIEWS                     567
TAG1                      645
TAG2                      645
GENDER                    567
MODEL                   98521
CODI_URL                    0
ITEM_URL                    0
id                        567
category_1                645
category_2                645
brand                     567
gender                    567
outfit_tags               567
img_url                   575
rating                  23008
codi_number               567
codi_tags                 567
codi_upload_date          567
codi_number_of_views      567
dtype: int64


,LEVEL,ITEM_ID,CODI_ID,STYLE,DATE,VIEWS,TAG1,TAG2,GENDER,MODEL,CODI_URL,ITEM_URL
0,1,3450087,35815,americancasual,NaN,NaN,NaN,NaN,NaN,NaN,https://www.musinsa.com/app/styles/views/35815...,https://www.musinsa.com/app/goods/3450087
1,1,3540430,35815,americancasual,2023.09.14,318.0,상의,맨투맨/스웨트셔츠,"남, 여",NaN,https://www.musinsa.com/app/styles/views/35815...,https://www.musinsa.com/app/goods/3540430
2,1,3227713,35815,americancasual,2023.09.01,974.0,스커트,미니스커트,여,NaN,https://www.musinsa.com/app/styles/views/35815...,https://www.musinsa.com/app/goods/3227713
3,1,3035852,35815,americancasual,2023.09.11,547.0,양말/레그웨어,양말,"남, 여",NaN,https://www.musinsa.com/app/styles/views/35815...,https://www.musinsa.com/app/goods/3035852
4,1,3429258,35815,americancasual,NaN,NaN,NaN,NaN,NaN,NaN,https://www.musinsa.com/app/styles/views/35815...,https://www.musinsa.com/app/goods/3429258
...,...,...,...,...,...,...,...,...,...,...,...,...
98516,1,1290270,21087,girlish,2021.05.21,1748.0,스포츠/용품,아우터,남,NaN,https://www.musinsa.com/app/styles/views/21087...,https://www.musinsa.com/app/goods/1290270
98517,1,1771289,21087,girlish,2021.05.21,1748.0,모자,버킷/사파리햇,남,NaN,https://www.musinsa.com/app/styles/views/21087...,https://www.musinsa.com/app/goods/1771289
98518,1,1771276,20947,girlish,2021.05.04,2957.0,모자,버킷/사파리햇,남,NaN,https://www.musinsa.com/app/styles/views/20947...,https://www.musinsa.com/app/goods/1771276
98519,1,1866392,20947,girlish,2021.05.04,2957.0,가방,백팩,남,NaN,https://www.musinsa.com/app/styles/views/20947...,https://www.musinsa.com/app/goods/1866392


#### 🍟 임시 통합본(2/2)

In [6]:
df_concat2 = pd.concat([df_level0, df_level1_db], ignore_index=True, axis=0)

df_concat2.insert(7, 'TAG2_', df_concat2['TAG2'])
df_concat2.insert(8, 'GENDER_', df_concat2['GENDER'])
df_concat2.drop(['TAG2', 'GENDER'], axis=1, inplace=True)
df_concat2.rename(columns={'TAG2_': 'TAG2'}, inplace=True)
df_concat2.rename(columns={'GENDER_': 'GENDER'}, inplace=True)

print(f"\x1b[33m\"LEVEL 0 데이터 ITEM_ID 처리...\"\x1b[0m")
tmp_dict = dict()
for codi, item in zip(df_concat2['CODI_ID'], df_concat2['ITEM_ID']):
    if item is not np.nan:
        # if codi == 35815:
        #     print(item)
        if codi in list(tmp_dict.keys()): 
            tmp_dict[codi] = tmp_dict[codi] + ',' + str(item)
        else:
            tmp_dict[codi] = str(item)
dict_key = list(tmp_dict.keys())
for idx, item in tqdm(enumerate(df_concat2['CODI_ID'])):
    if item in dict_key and df_concat2['ITEM_ID'][idx] is np.nan:
        df_concat2.iloc[idx, 1] = tmp_dict[item]
print("\x1b[32m\"LEVEL 0 데이터 ITEM_ID 처리 완료\"\x1b[0m")

df_concat2

"LEVEL 0 데이터 ITEM_ID 처리..."


129285it [00:04, 26118.90it/s]

"LEVEL 0 데이터 ITEM_ID 처리 완료"


,LEVEL,ITEM_ID,CODI_ID,STYLE,DATE,VIEWS,TAG1,TAG2,GENDER,MODEL,CODI_URL,ITEM_URL
0,0,"3450087,3540430,3227713,3035852,3429258",35815,americancasual,23.09.27,조회 607,스포티 바이브,NaN,NaN,모델 : 송혜원,https://www.musinsa.com/app/styles/views/35815...,NaN
1,0,"3451226,3524050,3523872,3431905,3450111,3450134",36233,americancasual,23.09.27,조회 587,시티보이 룩,NaN,NaN,모델 : 진킴,https://www.musinsa.com/app/styles/views/36233...,NaN
2,0,"3484627,3493988,3453009,3413269,3453042,3463945",36228,americancasual,23.09.27,조회 839,americancasual,NaN,NaN,모델 : 진킴,https://www.musinsa.com/app/styles/views/36228...,NaN
3,0,"3153140,3457629,3542129,2247148,3461469,3497070",36082,americancasual,23.09.21,조회 430,오버올 스타일,NaN,NaN,모델 : 김진령,https://www.musinsa.com/app/styles/views/36082...,NaN
4,0,"3561777,3483411,3430632,3152009,3431911,3479540",36081,americancasual,23.09.21,조회 925,프레피 룩,NaN,NaN,모델 : 김진령,https://www.musinsa.com/app/styles/views/36081...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
129280,1,1290270,21087,girlish,2021.05.21,1748.0,스포츠/용품,아우터,남,NaN,https://www.musinsa.com/app/styles/views/21087...,https://www.musinsa.com/app/goods/1290270
129281,1,1771289,21087,girlish,2021.05.21,1748.0,모자,버킷/사파리햇,남,NaN,https://www.musinsa.com/app/styles/views/21087...,https://www.musinsa.com/app/goods/1771289
129282,1,1771276,20947,girlish,2021.05.04,2957.0,모자,버킷/사파리햇,남,NaN,https://www.musinsa.com/app/styles/views/20947...,https://www.musinsa.com/app/goods/1771276
129283,1,1866392,20947,girlish,2021.05.04,2957.0,가방,백팩,남,NaN,https://www.musinsa.com/app/styles/views/20947...,https://www.musinsa.com/app/goods/1866392


## 📌 아이템 조회
* 추론단계에서 인덱스가 나왔을 때, 여기서 맵핑하여 아이템을 조회합니다.

In [24]:
# print(f"TAG1: {Counter(list(df_concat2['TAG1']))}")
# print(f"TAG2: {Counter(list(df_concat2['TAG2']))}")

### 🍔 중분류 아이템 조회
* ex) 미니스커트 조회

In [17]:
view_item_midclass = df_concat2[df_concat2['TAG2'] == '미니스커트'].reset_index(drop=True)
slicing_startnum = 0 # 시작 인덱스
slicing_endnum = 1 # 종료 인덱스

print("\x1b[32m\"⬇⬇⬇ 아이템(중분류 단위) ⬇⬇⬇\"\x1b[0m")
print(f"\x1b[33m\"전체 Shape: {view_item_midclass.shape}, Snippet Shape: {view_item_midclass.loc[slicing_startnum:slicing_endnum].shape}\"\x1b[0m")
display(view_item_midclass.loc[slicing_startnum:slicing_endnum])

"⬇⬇⬇ 아이템(중분류 단위) ⬇⬇⬇"
"전체 Shape: (1416, 12), Snippet Shape: (2, 12)"


,LEVEL,ITEM_ID,CODI_ID,STYLE,DATE,VIEWS,TAG1,TAG2,GENDER,MODEL,CODI_URL,ITEM_URL
0,1,3227713,35815,americancasual,2023.09.01,974.0,스커트,미니스커트,여,NaN,https://www.musinsa.com/app/styles/views/35815...,https://www.musinsa.com/app/goods/3227713
1,1,3254190,34845,americancasual,2023.07.26,1156.0,스커트,미니스커트,여,NaN,https://www.musinsa.com/app/styles/views/34845...,https://www.musinsa.com/app/goods/3254190


### 🍔 특정 아이템 조회
* ITEM_ID를 입력하여 조회합니다.

In [131]:
view_item = df_concat2[df_concat2['ITEM_ID'] == 1507634]

print("\x1b[32m\"⬇⬇⬇ 특정 아이템 ⬇⬇⬇\"\x1b[0m")
print(f"\x1b[33m\"Shape: {view_item.shape}\"\x1b[0m")
display(view_item)

"⬇⬇⬇ 특정 아이템 ⬇⬇⬇"
"Shape: (1, 12)"


,LEVEL,ITEM_ID,CODI_ID,STYLE,DATE,VIEWS,TAG1,TAG2,GENDER,MODEL,CODI_URL,ITEM_URL
62274,1,1507634,17389,americancasual,2020.08.06,1545.0,상의,반소매 티셔츠,여,NaN,https://www.musinsa.com/app/styles/views/17389...,https://www.musinsa.com/app/goods/1507634


## 📌 Light-GCN 학습 데이터셋 구축
* 상의 ➡ 하의: 'upper_class_list.txt', 'bottom_item_list.txt', 'train.txt', 'test.txt'
* 하의 ➡ 상의: 'bottom_class_list.txt', 'upper_item_list.txt', 'train.txt', 'test.txt'

### 🍔 공통 실행

In [7]:
concat_df = copy.deepcopy(df_concat2)
print(concat_df.shape)
display(concat_df.tail(1))
print(f"\x1b[33m\" drop 전, null 체크 \"\x1b[0m")
print(concat_df.isnull().sum())

# TAG1/ gender 에서 null 행 삭제
concat_df.dropna(subset=["TAG1"], inplace=True)
concat_df.dropna(subset=["GENDER"], inplace=True)

concat_df['TAG2'] = concat_df['TAG2'].str.replace(' ', '')
concat_df['GENDER'] = concat_df['GENDER'].str.replace(' ', '')

print(f"\x1b[32m\" drop 후, null 체크 \"\x1b[0m")
print(concat_df.isnull().sum())

# 상하의를 분류
con1 = concat_df['LEVEL'] == 1
con2 = concat_df["TAG1"].str.contains("상의", regex= True)
con3 = concat_df["TAG1"].str.contains("바지|스커트", regex= True)
 
df_up = copy.deepcopy(concat_df[con1 & con2])
df_down = copy.deepcopy(concat_df[con1 & con3])

(129285, 12)


,LEVEL,ITEM_ID,CODI_ID,STYLE,DATE,VIEWS,TAG1,TAG2,GENDER,MODEL,CODI_URL,ITEM_URL
129284,1,969541,20947,girlish,2020.07.15,2369.0,스니커즈,패션스니커즈화,"남, 여",NaN,https://www.musinsa.com/app/styles/views/20947...,https://www.musinsa.com/app/goods/969541


" drop 전, null 체크 "
LEVEL           0
ITEM_ID      3800
CODI_ID         0
STYLE           0
DATE          567
VIEWS         567
TAG1          645
TAG2        31409
GENDER      31331
MODEL       98521
CODI_URL        0
ITEM_URL    30764
dtype: int64
" drop 후, null 체크 "
LEVEL           0
ITEM_ID         0
CODI_ID         0
STYLE           0
DATE            0
VIEWS           0
TAG1            0
TAG2            0
GENDER          0
MODEL       97876
CODI_URL        0
ITEM_URL        0
dtype: int64


### 🍔 하의 ➡ 상의 raw 파일 만들기

In [8]:
print(f"\x1b[33m\" 상의 GENDER 컬럼: {df_up['GENDER'].unique()} \"\x1b[0m")
print(f"\x1b[33m\" 하의 GENDER 컬럼: {df_down['GENDER'].unique()} \"\x1b[0m")

# gender에 라이프 제거 (하의)
df_down[df_down['GENDER'] == '라이프']
df_down["GENDER"] = df_down["GENDER"].str.replace("라이프", "남,여")

print(f"\x1b[32m\" 상의 GENDER 컬럼: {df_up['GENDER'].unique()} \"\x1b[0m")
print(f"\x1b[32m\" 하의 GENDER 컬럼: {df_down['GENDER'].unique()} \"\x1b[0m")

" 상의 GENDER 컬럼: ['남,여' '여' '남'] "
" 하의 GENDER 컬럼: ['여' '남' '남,여' '라이프'] "
" 상의 GENDER 컬럼: ['남,여' '여' '남'] "
" 하의 GENDER 컬럼: ['여' '남' '남,여'] "


⬇⬇⬇ "CATEGORY1" 가능한 경우의 수(<b>필수 실행</b>)

In [9]:
df_down["CATEGORY1"] = df_down["STYLE"] + df_down["TAG2"] + df_down["GENDER"]
df_down["CATEGORY1"] = df_down["CATEGORY1"].str.replace(" ", "")

down_cat_list = list(df_down['CATEGORY1'].unique())

print(f"Shape: {df_down.shape}, Unique of CATEGORY1: ({len(df_down['CATEGORY1'].unique())})")
display(df_down.tail(1))

Shape: (14900, 13), Unique of CATEGORY1: (245)


,LEVEL,ITEM_ID,CODI_ID,STYLE,DATE,VIEWS,TAG1,TAG2,GENDER,MODEL,CODI_URL,ITEM_URL,CATEGORY1
129277,1,1920528,21185,girlish,2021.05.28,1607.0,바지,트레이닝/조거팬츠,남,NaN,https://www.musinsa.com/app/styles/views/21185...,https://www.musinsa.com/app/goods/1920528,girlish트레이닝/조거팬츠남


#### 🍟 upper_item_list.txt 만들기

In [10]:
before_txt_list = ["org_id remap_id"]
upper_item_dict = dict()

upper_itemid_list = df_up['ITEM_ID'].unique()
upper_itemid_list.sort()

for idx, item in enumerate(upper_itemid_list):
    before_txt_list.append(f"\r{item} {idx}")
    upper_item_dict[idx] = item

if not os.path.exists(os.path.join("..", "data", "BottomToUpper", "raw")):
    os.makedirs(os.path.join("..", "data", "BottomToUpper", "raw"))

upper_item_txt = " ".join(before_txt_list)

filename = os.path.join("..", "data", "BottomToUpper", "raw","upper_item_list.txt")

with open(f"{filename}", "w") as file:
    file.write(upper_item_txt)
    file.close()

print(f"\x1b[32m\"{filename} 저장되었습니다.\"\x1b[0m")

"../data/BottomToUpper/raw/upper_item_list.txt 저장되었습니다."


#### 🍟 train.txt 만들기

In [11]:
def get_key(val):
    for key, value in upper_item_dict.items():
        if val == value:
            return key
    return "key doesn't exist"

# df_down CATEGORY1 컬럼값별로 ITEM_ID를 모아서 리스트로 만들기
upper_itemlist = list()
noitem_list = list()

forindex = 0
for idx, item in enumerate(down_cat_list): # 245번 돌 것
    tmp_list = []
    for idx2, item2 in enumerate(set(list(df_down[df_down['CATEGORY1'] == item]['CODI_ID']))):
        for item3 in df_up[df_up['CODI_ID'] == item2]['ITEM_ID']:
            tmp_list.append(get_key(item3))
    if len(tmp_list) == 0:
        noitem_list.append(forindex)
    elif len(tmp_list) > 0:
        upper_itemlist.append(f"{forindex} {str(tmp_list).replace('[', '').replace(']', '').replace(',', '')}")
        forindex += 1

train_txt = "\n".join(upper_itemlist)
filename = os.path.join("..", "data", "BottomToUpper", "raw", "train.txt")

with open(f"{filename}", "w") as file:
    file.write(train_txt)
    file.close()

print(f"\x1b[32m\"{filename} 저장되었습니다.\"\x1b[0m")
print(f"{'-' * 50}")
print(f"CATEGORY1 : {len(down_cat_list)}개")
print(f"비어있는 인덱스[{len(noitem_list)}개] {noitem_list}")
print(f"ex) 인덱스 {noitem_list[0]}번 {down_cat_list[noitem_list[0]]} 제거... ")
print(f"비어있는 인덱스 제거 후: {len(down_cat_list)} - {len(noitem_list)} = {len(upper_itemlist)} 개")

"../data/BottomToUpper/raw/train.txt 저장되었습니다."
--------------------------------------------------
CATEGORY1 : 245개
비어있는 인덱스[16개] [76, 117, 137, 138, 140, 158, 159, 173, 175, 177, 186, 197, 211, 214, 216, 227]
ex) 인덱스 76번 dandy점프슈트/오버올남,여 제거... 
비어있는 인덱스 제거 후: 245 - 16 = 229 개


#### 🍟 bottom_class_list.txt 만들기
* 하의를 Style, 성별, 중분류 특성을 참고하여 분류

In [12]:
tmp_list = ["org_id remap_id"]

forindex = 0
for idx, item in enumerate(down_cat_list):
    if idx not in noitem_list:
        tmp_list.append(f"\r{item} {forindex}")
        forindex +=1

bottom_class_txt = " ".join(tmp_list)

filename = os.path.join("..", "data", "BottomToUpper", "raw", "bottom_class_list.txt")

with open(f"{filename}", "w") as file:
    file.write(bottom_class_txt)
    file.close()

print(f"\x1b[32m\"{filename} 저장되었습니다.\"\x1b[0m")

"../data/BottomToUpper/raw/bottom_class_list.txt 저장되었습니다."


#### 🍟 test.txt 만들기 (train.txt 복사 해서 변형하세요)

In [13]:
original_file = os.path.join("..", "data", "BottomToUpper", "raw", "train.txt")
copy_file = os.path.join("..", "data", "BottomToUpper", "raw", "test.txt")

try:
    with open(original_file, 'r') as of:
        tmp_txt = of.read()
        
        with open(copy_file, 'w') as cf:
            cf.write(tmp_txt)
        print(f"\x1b[32m\"{copy_file} 저장되었습니다.\"\x1b[0m")
except FileNotFoundError:
    print('파일을 찾을 수 없습니다.')
except Exception as e:
    print('파일 복사 중 오류가 발생했습니다:', e)


"../data/BottomToUpper/raw/test.txt 저장되었습니다."


### 🍔 상의 ➡ 하의 raw 파일 만들기

⬇⬇⬇ "CATEGORY1" 가능한 경우의 수(<b>필수 실행</b>)

In [14]:
df_up["CATEGORY1"] = df_up["STYLE"] + df_up["TAG2"] + df_up["GENDER"]
df_up["CATEGORY1"] = df_up["CATEGORY1"].str.replace(" ", "")

up_cat_list = list(df_up['CATEGORY1'].unique())

print(f"Shape: {df_up.shape}, Unique of CATEGORY1: ({len(df_up['CATEGORY1'].unique())})")
display(df_up.tail(1))

Shape: (15935, 13), Unique of CATEGORY1: (259)


,LEVEL,ITEM_ID,CODI_ID,STYLE,DATE,VIEWS,TAG1,TAG2,GENDER,MODEL,CODI_URL,ITEM_URL,CATEGORY1
129274,1,1947912,21323,girlish,2021.06.08,1657.0,상의,셔츠/블라우스,남,NaN,https://www.musinsa.com/app/styles/views/21323...,https://www.musinsa.com/app/goods/1947912,girlish셔츠/블라우스남


#### 🍟 bottom_item_list.txt 만들기

In [15]:
before_txt_list = ["org_id remap_id"]
bottom_item_dict = dict()

bottom_itemid_list = df_down['ITEM_ID'].unique()
bottom_itemid_list.sort()

for idx, item in enumerate(bottom_itemid_list):
    before_txt_list.append(f"\r{item} {idx}")
    bottom_item_dict[idx] = item

bottom_item_txt = " ".join(before_txt_list)

if not os.path.exists(os.path.join("..", "data", "UpperToBottom", "raw")):
    os.makedirs(os.path.join("..", "data", "UpperToBottom", "raw"))

filename = os.path.join("..", "data", "UpperToBottom", "raw", "bottom_item_list.txt")

with open(f"{filename}", "w") as file:
    file.write(bottom_item_txt)
    file.close()

print(f"\x1b[32m\"{filename} 저장되었습니다.\"\x1b[0m")

"../data/UpperToBottom/raw/bottom_item_list.txt 저장되었습니다."


#### 🍟 train.txt 만들기

In [16]:
def get_key(val):
    for key, value in bottom_item_dict.items():
        if val == value:
            return key
    return "key doesn't exist"

# df_up CATEGORY1 컬럼값별로 ITEM_ID를 모아서 리스트로 만들기
bottom_itemlist = list()
noitem_list = list()

forindex = 0
for idx, item in enumerate(up_cat_list): 
    tmp_list = []
    for idx2, item2 in enumerate(set(list(df_up[df_up['CATEGORY1'] == item]['CODI_ID']))):
        for item3 in df_down[df_down['CODI_ID'] == item2]['ITEM_ID']:
            tmp_list.append(get_key(item3))
    if len(tmp_list) == 0:
        noitem_list.append(forindex)
    elif len(tmp_list) > 0:
        bottom_itemlist.append(f"{forindex} {str(tmp_list).replace('[', '').replace(']', '').replace(',', '')}")
        forindex += 1

train_txt = "\n".join(bottom_itemlist)
filename = os.path.join("..", "data", "UpperToBottom", "raw", "train.txt")

with open(f"{filename}", "w") as file:
    file.write(train_txt)
    file.close()

print(f"\x1b[32m\"{filename} 저장되었습니다.\"\x1b[0m")
print(f"{'-' * 50}")
print(f"CATEGORY1 : {len(up_cat_list)}개")
print(f"비어있는 인덱스[{len(noitem_list)}개] {noitem_list}")
print(f"ex) 인덱스 {noitem_list[0]}번 {up_cat_list[noitem_list[0]]} 제거... ")
print(f"비어있는 인덱스 제거 후: {len(up_cat_list)} - {len(noitem_list)} = {len(bottom_itemlist)} 개")

"../data/UpperToBottom/raw/train.txt 저장되었습니다."
--------------------------------------------------
CATEGORY1 : 259개
비어있는 인덱스[12개] [149, 152, 154, 163, 181, 182, 186, 186, 230, 236, 241, 246]
ex) 인덱스 149번 golf반소매티셔츠남,여 제거... 
비어있는 인덱스 제거 후: 259 - 12 = 247 개


#### 🍟 upper_class_list.txt 만들기
* 상의를 Style, 성별, 중분류 특성을 참고하여 분류

In [17]:
tmp_list = ["org_id remap_id"]

forindex = 0
for idx, item in enumerate(up_cat_list):
    if idx not in noitem_list:
        tmp_list.append(f"\r{item} {forindex}")
        forindex +=1

upper_class_txt = " ".join(tmp_list)

filename = os.path.join("..", "data", "UpperToBottom", "raw", "upper_class_list.txt")

with open(f"{filename}", "w") as file:
    file.write(upper_class_txt)
    file.close()

print(f"\x1b[32m\"{filename} 저장되었습니다.\"\x1b[0m")

"../data/UpperToBottom/raw/upper_class_list.txt 저장되었습니다."


#### 🍟 test.txt 만들기 (train.txt 복사 해서 변형하세요)

In [18]:
original_file = os.path.join("..", "data", "UpperToBottom", "raw", "train.txt")
copy_file = os.path.join("..", "data", "UpperToBottom", "raw", "test.txt")

try:
    with open(original_file, 'r') as of:
        tmp_txt = of.read()
        
        with open(copy_file, 'w') as cf:
            cf.write(tmp_txt)
        print(f"\x1b[32m\"{copy_file} 저장되었습니다.\"\x1b[0m")
except FileNotFoundError:
    print('파일을 찾을 수 없습니다.')
except Exception as e:
    print('파일 복사 중 오류가 발생했습니다:', e)


"../data/UpperToBottom/raw/test.txt 저장되었습니다."


In [31]:
! ls /opt/conda/lib/python3.10/site-packages/torch_geometric/

__init__.py	 data		  explain	  logging.py   testing
__pycache__	 datasets	  graphgym	  nn	       transforms
backend.py	 debug.py	  home.py	  profile      typing.py
compile.py	 deprecation.py   io		  resolver.py  utils
config_store.py  distributed	  lazy_loader.py  sampler      visualization
contrib		 experimental.py  loader	  seed.py      warnings.py


## 📌 Light-GCN Train

하의 ➡ 상의 전용 train

In [19]:
def train():
    total_loss = total_examples = 0

    for index in tqdm(train_loader):
        # Sample positive and negative labels.
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_books,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(data.edge_index, edge_label_index).chunk(2) # model은 여기에

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples


@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    for start in range(0, num_users, batch_size):
        end = start + batch_size
        logits = user_emb[start:end] @ book_emb.t()

        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
               train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                     data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                            num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

    return precision / total_examples, recall / total_examples

### 🍔 하의 ➡ 상의

In [20]:
path = osp.join('/', 'step1','data', 'BottomToUpper')
dataset = BottomToUpper(path)
data = dataset[0]
num_users, num_books = data['bottom'].num_nodes, data['upper'].num_nodes
data = data.to_homogeneous().to(device)

# Use all message passing edges as training labels:
batch_size = 2048
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

"하의류(스타일, 중분류, 성별) 클래스 ➡ 상의아이템 추천 datasets 구축."


In [21]:
model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=64,
    num_layers=4,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
positive_edge_label_idx_list = []
negative_edge_label_idx_list = []
edge_label_index_list = []

for index in tqdm(train_loader):
    # Sample positive and negative labels.
    pos_edge_label_index = train_edge_label_index[:, index]
    neg_edge_label_index = torch.stack([
        pos_edge_label_index[0],
        torch.randint(num_users, num_users + num_books,
                      (index.numel(), ), device=device)
    ], dim=0)
    edge_label_index = torch.cat([
        pos_edge_label_index,
        neg_edge_label_index,
    ], dim=1)
    positive_edge_label_idx_list.append(pos_edge_label_index)
    negative_edge_label_idx_list.append(neg_edge_label_index)
    edge_label_index_list.append(edge_label_index)

print(f"edge_label_index_list 전치: {len(edge_label_index_list[0].T)}, batchsize: {int(len(edge_label_index_list[0].T) / 2)} ")

100%|██████████| 6/6 [00:00<00:00, 1974.72it/s]

edge_label_index_list 전치: 4096, batchsize: 2048 


학습 전 Interaction 확률 확인
* 완전 랜덤인 것을 확인할 수 있음

In [25]:
model.eval()
with torch.no_grad():
    pred = model.predict_link(edge_index = data.edge_index, prob=True)
pred

tensor([0.5003, 0.5002, 0.5003,  ..., 0.5002, 0.5001, 0.5012], device='cuda:0')

#### 🍟 학습

In [23]:
epoch = 100

for epoch in range(0, epoch):
    loss = train()
    precision, recall = test(k=20)
    print(f'Epoch: {epoch+1:03d}, Loss: {loss:.4f}, Precision@20: '
          f'{precision:.4f}, Recall@20: {recall:.4f}')

100%|██████████| 6/6 [00:00<00:00, 34.08it/s]


Epoch: 001, Loss: 0.6924, Precision@20: 0.0000, Recall@20: 0.0000


100%|██████████| 6/6 [00:00<00:00, 317.19it/s]


Epoch: 002, Loss: 0.6886, Precision@20: 0.0000, Recall@20: 0.0000


100%|██████████| 6/6 [00:00<00:00, 324.66it/s]


...

Epoch: 098, Loss: 0.0356, Precision@20: 0.0000, Recall@20: 0.0000


100%|██████████| 6/6 [00:00<00:00, 342.32it/s]


Epoch: 099, Loss: 0.0365, Precision@20: 0.0000, Recall@20: 0.0000


100%|██████████| 6/6 [00:00<00:00, 347.98it/s]

Epoch: 100, Loss: 0.0385, Precision@20: 0.0000, Recall@20: 0.0000


#### 🍟 추론

In [24]:
model.eval()
with torch.no_grad():
    pred = model.predict_link(edge_index = data.edge_index, prob=True)
pred

tensor([0.9775, 0.9778, 0.9740,  ..., 0.9780, 0.9762, 0.7795], device='cuda:0')

In [28]:
bottom_to_upper = zip(data.edge_index[0], data.edge_index[1], pred)

for index0, index1, percent in bottom_to_upper:
    
    print(index0, index1, percent)

tensor(0, device='cuda:0') tensor(3419, device='cuda:0') tensor(0.9832, device='cuda:0')
tensor(0, device='cuda:0') tensor(3214, device='cuda:0') tensor(0.9817, device='cuda:0')
tensor(0, device='cuda:0') tensor(2741, device='cuda:0') tensor(0.9821, device='cuda:0')
tensor(0, device='cuda:0') tensor(11453, device='cuda:0') tensor(0.9552, device='cuda:0')
tensor(0, device='cuda:0') tensor(3242, device='cuda:0') tensor(0.9816, device='cuda:0')
tensor(0, device='cuda:0') tensor(3043, device='cuda:0') tensor(0.9827, device='cuda:0')
tensor(0, device='cuda:0') tensor(3252, device='cuda:0') tensor(0.9838, device='cuda:0')
tensor(0, device='cuda:0') tensor(10534, device='cuda:0') tensor(0.9673, device='cuda:0')
tensor(0, device='cuda:0') tensor(3298, device='cuda:0') tensor(0.9591, device='cuda:0')
tensor(1, device='cuda:0') tensor(11355, device='cuda:0') tensor(0.8496, device='cuda:0')
tensor(2, device='cuda:0') tensor(11432, device='cuda:0') tensor(0.8220, device='cuda:0')
tensor(2, device=

In [25]:
filename = os.path.join("..", "data", "BottomToUpper", "raw", "bottom_class_list.txt")

with open(f"{filename}", "r") as file:
    bottom_to_upper_contents = file.read()
tmp_list = bottom_to_upper_contents.split('\n')

before_df = []
for i in tmp_list:
    tmp_j = []
    for j in i.split(' '):
        if j != '':
            tmp_j.append(j)
    before_df.append(tmp_j)
before_df = before_df[1:]
df_bottom_class = pd.DataFrame(before_df, columns=['bottom_class_org_id', 'bottom_class_remap_id'])
df_bottom_class['bottom_class_remap_id'] = df_bottom_class['bottom_class_remap_id'].astype('int64')
df_bottom_class.tail(1)


,bottom_class_org_id,bottom_class_remap_id
228,"girlish슈트팬츠/슬랙스남,여",228


In [26]:
filename = os.path.join("..", "data", "BottomToUpper", "raw", "upper_item_list.txt")

with open(f"{filename}", "r") as file:
    bottom_to_upper_contents = file.read()
tmp_list = bottom_to_upper_contents.split('\n')

before_df = []
for i in tmp_list:
    tmp_j = []
    for j in i.split(' '):
        if j != '':
            tmp_j.append(j)
    before_df.append(tmp_j)
before_df = before_df[1:]
df_upper_item = pd.DataFrame(before_df, columns=['upper_item_org_id', 'upper_item_remap_id'])
df_upper_item['upper_item_remap_id'] = df_upper_item['upper_item_remap_id'].astype('int64')
df_upper_item.tail(1)

,upper_item_org_id,upper_item_remap_id
11229,3561898,11229


In [27]:
item_url = pd.DataFrame(concat_df[['ITEM_ID','ITEM_URL']])
item_url.columns = ['upper_item_org_id', 'ITEM_URL']

item_url['upper_item_org_id'] = item_url['upper_item_org_id'].astype('int64')

print(item_url.shape)
item_url.tail(1)

(97876, 2)


,upper_item_org_id,ITEM_URL
129284,969541,https://www.musinsa.com/app/goods/969541


In [106]:
item_url.dtypes

upper_item_org_id    object
ITEM_URL             object
dtype: object

In [29]:
df_pred_upper = pd.DataFrame()
df_pred_upper['edge_index0'] = data.edge_index[0].tolist()
df_pred_upper['edge_index1'] = data.edge_index[1].tolist()
df_pred_upper['prob'] = pred.tolist()

df_pred_upper = df_pred_upper[df_pred_upper['edge_index0'] < 230]

df_pred_upper['upper_item_remap_id'] = df_pred_upper['edge_index1'] - 230
df_pred_upper.dtypes
merge_df_upper = pd.merge(df_pred_upper, df_upper_item, on = 'upper_item_remap_id', how='left')
merge_df_upper['bottom_class_remap_id'] = merge_df_upper['edge_index0']
merge_df_upper = pd.merge(merge_df_upper, df_bottom_class, on = 'bottom_class_remap_id', how='left')
merge_df_upper['upper_item_org_id'] = merge_df_upper['upper_item_org_id'].astype('int64')
merge_df_upper = pd.merge(merge_df_upper, item_url, on = 'upper_item_org_id', how = 'left')
merge_df_upper.to_csv(os.path.join("..", "data", "bottom_to_upper.csv"),index=True, encoding='utf-8')

print(merge_df_upper.shape)

display(merge_df_upper)
print(merge_df_upper.dtypes)
# df_pred_upper.head(10)


(16010, 8)


,edge_index0,edge_index1,prob,upper_item_remap_id,upper_item_org_id,bottom_class_remap_id,bottom_class_org_id,ITEM_URL
0,0,3419,0.977459,3189,1507634,0,americancasual미니스커트여,https://www.musinsa.com/app/goods/1507634
1,0,3214,0.977764,2984,1456146,0,americancasual미니스커트여,https://www.musinsa.com/app/goods/1456146
2,0,2741,0.973962,2511,1364514,0,americancasual미니스커트여,https://www.musinsa.com/app/goods/1364514
3,0,11453,0.961531,11223,3540429,0,americancasual미니스커트여,https://www.musinsa.com/app/goods/3540429
4,0,3242,0.978662,3012,1461400,0,americancasual미니스커트여,https://www.musinsa.com/app/goods/1461400
...,...,...,...,...,...,...,...,...
16005,227,11161,0.977972,10931,3374934,227,girlish숏팬츠남,https://www.musinsa.com/app/goods/3374934
16006,227,11161,0.977972,10931,3374934,227,girlish숏팬츠남,https://www.musinsa.com/app/goods/3374934
16007,227,1349,0.977984,1119,1035192,227,girlish숏팬츠남,https://www.musinsa.com/app/goods/1035192
16008,227,9662,0.976220,9432,3134283,227,girlish숏팬츠남,https://www.musinsa.com/app/goods/3134283


edge_index0                int64
edge_index1                int64
prob                     float64
upper_item_remap_id        int64
upper_item_org_id          int64
bottom_class_remap_id      int64
bottom_class_org_id       object
ITEM_URL                  object
dtype: object


### 🍔 상의 ➡ 하의

In [34]:
def train2():
    total_loss = total_examples = 0

    for index in tqdm(train_loader):
        # Sample positive and negative labels.
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_books,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(data2.edge_index, edge_label_index).chunk(2) # model은 여기에

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples


@torch.no_grad()
def test2(k: int):
    emb = model.get_embedding(data2.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    for start in range(0, num_users, batch_size):
        end = start + batch_size
        logits = user_emb[start:end] @ book_emb.t()

        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
               train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data2.edge_label_index[0] >= start) &
                (data2.edge_label_index[0] < end))
        ground_truth[data2.edge_label_index[0, mask] - start,
                     data2.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data2.edge_label_index[0, mask] - start,
                            num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

    return precision / total_examples, recall / total_examples

In [13]:
#! rm -r /step1/data/UpperToBottom/processed

In [30]:
path = osp.join('/', 'step1','data', 'UpperToBottom')
dataset = UpperToBottom(path)
data2 = dataset[0]
num_users, num_books = data2['upper'].num_nodes, data2['bottom'].num_nodes
data2 = data2.to_homogeneous().to(device)

# Use all message passing edges as training labels:
batch_size = 2048
mask = data2.edge_index[0] < data2.edge_index[1]
train_edge_label_index = data2.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

In [31]:
model = LightGCN(
    num_nodes=data2.num_nodes,
    embedding_dim=64,
    num_layers=4,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [32]:
positive_edge_label_idx_list = []
negative_edge_label_idx_list = []
edge_label_index_list = []

for index in tqdm(train_loader):
    # Sample positive and negative labels.
    pos_edge_label_index = train_edge_label_index[:, index]
    neg_edge_label_index = torch.stack([
        pos_edge_label_index[0],
        torch.randint(num_users, num_users + num_books,
                      (index.numel(), ), device=device)
    ], dim=0)
    edge_label_index = torch.cat([
        pos_edge_label_index,
        neg_edge_label_index,
    ], dim=1)
    positive_edge_label_idx_list.append(pos_edge_label_index)
    negative_edge_label_idx_list.append(neg_edge_label_index)
    edge_label_index_list.append(edge_label_index)

print(f"edge_label_index_list 전치: {len(edge_label_index_list[0].T)}, batchsize: {int(len(edge_label_index_list[0].T) / 2)} ")

100%|██████████| 6/6 [00:00<00:00, 2149.64it/s]

edge_label_index_list 전치: 4096, batchsize: 2048 


학습 전 interaction 확률 확인

In [33]:
model.eval()
with torch.no_grad():
    pred = model.predict_link(edge_index = data2.edge_index, prob=True)
pred

tensor([0.5002, 0.5002, 0.5002,  ..., 0.5002, 0.5004, 0.5006], device='cuda:0')

In [35]:
epoch = 100

for epoch in range(0, epoch):
    loss = train2()
    precision, recall = test2(k=20)
    print(f'Epoch: {epoch+1:03d}, Loss: {loss:.4f}, Precision@20: '
          f'{precision:.4f}, Recall@20: {recall:.4f}')

100%|██████████| 6/6 [00:00<00:00, 28.23it/s]


Epoch: 001, Loss: 0.6925, Precision@20: 0.0000, Recall@20: 0.0000


100%|██████████| 6/6 [00:00<00:00, 322.00it/s]


Epoch: 002, Loss: 0.6892, Precision@20: 0.0000, Recall@20: 0.0000


100%|██████████| 6/6 [00:00<00:00, 366.31it/s]


...

Epoch: 098, Loss: 0.0458, Precision@20: 0.0000, Recall@20: 0.0000


100%|██████████| 6/6 [00:00<00:00, 386.76it/s]


Epoch: 099, Loss: 0.0480, Precision@20: 0.0000, Recall@20: 0.0000


100%|██████████| 6/6 [00:00<00:00, 393.27it/s]

Epoch: 100, Loss: 0.0477, Precision@20: 0.0000, Recall@20: 0.0000


In [36]:
model.eval()
with torch.no_grad():
    pred = model.predict_link(edge_index = data2.edge_index, prob=True)
pred

tensor([0.9600, 0.9606, 0.9360,  ..., 0.9634, 0.9274, 0.8913], device='cuda:0')

In [43]:
filename = os.path.join("..", "data", "UpperToBottom", "raw", "upper_class_list.txt")

with open(f"{filename}", "r") as file:
    upper_to_bottom_contents = file.read()
tmp_list = upper_to_bottom_contents.split('\n')

before_df = []
for i in tmp_list:
    tmp_j = []
    for j in i.split(' '):
        if j != '':
            tmp_j.append(j)
    before_df.append(tmp_j)
before_df = before_df[1:]
df_upper_class = pd.DataFrame(before_df, columns=['upper_class_org_id', 'upper_class_remap_id'])
df_upper_class['upper_class_remap_id'] = df_upper_class['upper_class_remap_id'].astype('int64')
df_upper_class.tail(1)


,upper_class_org_id,upper_class_remap_id
247,"girlish피케/카라티셔츠남,여",247


In [44]:
filename = os.path.join("..", "data", "UpperToBottom", "raw", "bottom_item_list.txt")

with open(f"{filename}", "r") as file:
    upper_to_bottom_contents = file.read()
tmp_list = upper_to_bottom_contents.split('\n')

before_df = []
for i in tmp_list:
    tmp_j = []
    for j in i.split(' '):
        if j != '':
            tmp_j.append(j)
    before_df.append(tmp_j)
before_df = before_df[1:]
df_bottom_item = pd.DataFrame(before_df, columns=['bottom_item_org_id', 'bottom_item_remap_id'])
df_bottom_item['bottom_item_remap_id'] = df_bottom_item['bottom_item_remap_id'].astype('int64')
df_bottom_item.tail(1)

,bottom_item_org_id,bottom_item_remap_id
10146,3544470,10146


In [45]:
item_url = pd.DataFrame(concat_df[['ITEM_ID','ITEM_URL']])
item_url.columns = ['bottom_item_org_id', 'ITEM_URL']

item_url['bottom_item_org_id'] = item_url['bottom_item_org_id'].astype('int64')

print(item_url.shape)
item_url.tail(1)

(97876, 2)


,bottom_item_org_id,ITEM_URL
129284,969541,https://www.musinsa.com/app/goods/969541


In [51]:
merge_df_upper.isnull().sum()

edge_index0              0
edge_index1              0
prob                     0
bottom_item_remap_id     0
bottom_item_org_id      18
upper_class_remap_id     0
upper_class_org_id       0
dtype: int64

In [ ]:
merge_df_upper = merge_df_upper.dropna()

In [52]:
df_pred_bottom = pd.DataFrame()
df_pred_bottom['edge_index0'] = data2.edge_index[0].tolist()
df_pred_bottom['edge_index1'] = data2.edge_index[1].tolist()
df_pred_bottom['prob'] = pred.tolist()

df_pred_bottom = df_pred_bottom[df_pred_bottom['edge_index0'] < 230]

df_pred_bottom['bottom_item_remap_id'] = df_pred_bottom['edge_index1'] - 230

merge_df_upper = pd.merge(df_pred_bottom, df_bottom_item, on = 'bottom_item_remap_id', how='left')
merge_df_upper['upper_class_remap_id'] = merge_df_upper['edge_index0']
merge_df_upper = pd.merge(merge_df_upper, df_upper_class, on = 'upper_class_remap_id', how='left')

merge_df_upper = merge_df_upper.dropna()

merge_df_upper['bottom_item_org_id'] = merge_df_upper['bottom_item_org_id'].astype('int64')
merge_df_upper = pd.merge(merge_df_upper, item_url, on = 'bottom_item_org_id', how = 'left')
merge_df_upper.to_csv(os.path.join("..", "data", "upper_to_bottom.csv"),index=True, encoding='utf-8')

print(merge_df_upper.shape)

display(merge_df_upper)
print(merge_df_upper.dtypes)
# df_pred_upper.head(10)


(15174, 8)


,edge_index0,edge_index1,prob,bottom_item_remap_id,bottom_item_org_id,upper_class_remap_id,upper_class_org_id,ITEM_URL
0,0,9507,0.959954,9277,3245973,0,"americancasual맨투맨/스웨트셔츠남,여",https://www.musinsa.com/app/goods/3245973
1,0,1931,0.960567,1701,1325879,0,"americancasual맨투맨/스웨트셔츠남,여",https://www.musinsa.com/app/goods/1325879
2,0,8296,0.935963,8066,3058466,0,"americancasual맨투맨/스웨트셔츠남,여",https://www.musinsa.com/app/goods/3058466
3,0,9438,0.916834,9208,3230719,0,"americancasual맨투맨/스웨트셔츠남,여",https://www.musinsa.com/app/goods/3230719
4,0,10257,0.963593,10027,3442314,0,"americancasual맨투맨/스웨트셔츠남,여",https://www.musinsa.com/app/goods/3442314
...,...,...,...,...,...,...,...,...
15169,228,2476,0.929780,2246,1461214,228,gorpcore니트/스웨터여,https://www.musinsa.com/app/goods/1461214
15170,228,2893,0.956603,2663,1596414,228,gorpcore니트/스웨터여,https://www.musinsa.com/app/goods/1596414
15171,228,2142,0.981810,1912,1372793,228,gorpcore니트/스웨터여,https://www.musinsa.com/app/goods/1372793
15172,228,1876,0.953896,1646,1321700,228,gorpcore니트/스웨터여,https://www.musinsa.com/app/goods/1321700


edge_index0               int64
edge_index1               int64
prob                    float64
bottom_item_remap_id      int64
bottom_item_org_id        int64
upper_class_remap_id      int64
upper_class_org_id       object
ITEM_URL                 object
dtype: object
